In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from PIL import Image, ImageDraw, ImageFont

In [ ]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                  
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [ ]:
def draw_styled_landmarks(image, results,draw=True):
      if draw:
        # Draw face connections
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                                mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)) 
        # Draw pose connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                                mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) 
        #Draw left hand connections
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
        # Draw right hand connections  
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 

In [ ]:
# def extract_keypoints(results):
#     pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
#     face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
#     lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
#     rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
#     return np.concatenate([pose, face, lh, rh])

In [ ]:
def extract_keypointsL(results):
   
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([lh])

In [ ]:
def extract_keypointsR(results):
    
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([rh])

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('NumberR.h5')

In [ ]:
# colors = [(245,117,16), (117,245,16), (16,117,245)]
# def prob_viz(res, actions, input_frame, colors):
#     output_frame = input_frame.copy()
#     for num, prob in enumerate(res):
#         cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
#         cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)       
#     return output_frame

In [ ]:


def symbolRight(image, holistic):
    frame_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(frame_rgb)
    
    if results.right_hand_landmarks and results.pose_landmarks:
        left_hand_landmarks = results.right_hand_landmarks
        wrist_x = int(left_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST].x * image.shape[1])
        wrist_y = int(left_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST].y * image.shape[0])
        
        # Access the landmark for the base of the middle finger
        middle_finger_base_x = int(left_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_MCP].x * image.shape[1])
        middle_finger_base_y = int(left_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_MCP].y * image.shape[0])

        palm_center_x = int((wrist_x + middle_finger_base_x) / 2)
        palm_center_y = int((wrist_y + middle_finger_base_y) / 2)
        symbol_x = palm_center_x - 10
        symbol_y = palm_center_y - 20
        
        font = ImageFont.truetype('SuttonSignWriting.ttf', 80)
        font_color = (0, 0, 0)  # Default color (black)
        font_style = "normal"    # Default style (normal)
        
        if hasattr(font, "color"):
            font_color = font.color
        if hasattr(font, "style"):
            font_style = font.style
        
        pil_frame = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil_frame)
          
        selected_index = np.argmax(res)
        selected_hex_string = actions[selected_index]
        hex_int = int(selected_hex_string, 16)
        symbol = chr(hex_int)
        
        symbol_position = (symbol_x, symbol_y)
        
        draw.text(symbol_position, symbol, font=font, fill=font_color, anchor="mm", align="center", spacing=0)
        image = cv2.cvtColor(np.array(pil_frame), cv2.COLOR_RGB2BGR)
    return image


In [ ]:


def symbolLeftt(image, holistic):
    frame_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(frame_rgb)
    
    if results.left_hand_landmarks and results.pose_landmarks:
        left_hand_landmarks = results.left_hand_landmarks
        wrist_x = int(left_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST].x * image.shape[1])
        wrist_y = int(left_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST].y * image.shape[0])
        
        # Access the landmark for the base of the middle finger
        middle_finger_base_x = int(left_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_MCP].x * image.shape[1])
        middle_finger_base_y = int(left_hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_MCP].y * image.shape[0])

        palm_center_x = int((wrist_x + middle_finger_base_x) / 2)
        palm_center_y = int((wrist_y + middle_finger_base_y) / 2)
        symbol_x = palm_center_x - 10
        symbol_y = palm_center_y - 20
        
        font = ImageFont.truetype('SuttonSignWriting.ttf', 80)
        font_color = (0, 0, 0)  # Default color (black)
        font_style = "normal"    # Default style (normal)
        
        if hasattr(font, "color"):
            font_color = font.color
        if hasattr(font, "style"):
            font_style = font.style
        
        pil_frame = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil_frame)
        
        selected_index = np.argmax(res)
        selected_hex_string = actions[selected_index]
        hex_int = int(selected_hex_string, 16)
        symbol = chr(hex_int)
        
        symbol_position = (symbol_x, symbol_y)
        
        draw.text(symbol_position, symbol, font=font, fill=font_color, anchor="mm", align="center", spacing=0)
        image = cv2.cvtColor(np.array(pil_frame), cv2.COLOR_RGB2BGR)
    return image


In [ ]:

with open('NumberR.txt', 'r') as file:
    actions = np.array([line.strip().split(',')[0] for line in file.readlines()])
with open('NumberR.txt', 'r') as file:
    actions1 = np.array([line.strip().split(',')[1] for line in file.readlines()])

no_sequences = 10

# Videos are going to be 5 frames in length
sequence_length = 5
print(actions)
print(actions1)

In [ ]:
#detection
sequence = []
sentence = []
threshold = 0.5

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()
        frame = cv2.flip(frame,1)

        image, results = mediapipe_detection(frame, holistic)
        print(results)

        draw_styled_landmarks(image, results,draw=False)
            
        # 2. Prediction logic
        keypoints = extract_keypointsL(results)

        sequence.append(keypoints)
        sequence = sequence[-5:]
                
        if len(sequence) == 5:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
                
            if res[np.argmax(res)] > threshold: 
                frame_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = symbolRight(image,holistic)
                #image = symbolLeftt(image,holistic)    
            if len(sentence) > 0: 
                if actions1[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions1[np.argmax(res)])
            else:
                sentence.append(actions1[np.argmax(res)])                
            if len(sentence) > 5:
                sentence = sentence[-5:]
                
                # Viz probabilities
                #image = prob_viz(res, actions1, image, colors)
                    
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, sentence[-1], (3,30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 25, 255), 2, cv2.LINE_AA)                   
            # Show to screen
            cv2.imshow('OpenCV Feed', image)
            # Break
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()